In [1]:
import numpy as np
from keras.layers import (Lambda, Input, Reshape,
                          Dense, UpSampling2D,
                          Conv2D, Concatenate,
                          Flatten, MaxPool2D,)
from keras.losses import mse, mae, binary_crossentropy
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

import keras.backend as K

Using TensorFlow backend.


In [2]:
def settrainable(model, toset):
    for layer in model.layers:
        layer.trainable = toset
    model.trainable = toset

In [3]:
input_shape = (256,8,1) # (nfft,n_lookback,n_channels)
filters = [32,32,32]
layers = 3
kernel_size = [3,3]
pool_size = [2,2]
intermediate = 16
latent_dim = 50
n_classes = 8

inputs = Input(shape=input_shape)
x = inputs
for i in range(layers):
    x = Conv2D(filters[i],
               kernel_size=kernel_size,
               activation='relu',
               padding='same')(x)
    x = MaxPool2D(pool_size=pool_size)(x)


# shape info needed to build decoder model
shape = K.int_shape(x)

x = Flatten()(x)
# generate latent vector Q(z|X)
features = Dense(latent_dim, activation='linear',name='features')(x)
classes = Dense(n_classes, activation='softmax',name='classes')(x)

# no reparameterization trick here, since we will get the distribution
#   through adversarial training against a stocastic process we don't
#   need to sample here

encoder = Model(inputs, [features,classes], name='encoder')
encoder.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 8, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 8, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 4, 32)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 4, 32)   9248        max_pooling2d_1[0][0]            
_____________________________________

In [4]:
filters = [32,64,64]
upsampling_size = pool_size

latent_inputs = Input(shape=(latent_dim+n_classes,), name='decoder_input')
x = Dense(shape[1] * shape[2] * shape[3], activation='relu')(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)

for i in range(layers):
    x = Conv2D(filters=filters[i],
               kernel_size=kernel_size,
               activation='relu',
               padding='same')(x)
    x = UpSampling2D(size=upsampling_size)(x)

x = Conv2D(filters=1,
           kernel_size=kernel_size,
           activation='relu',
           padding='same',
           name='decoder_output')(x)

outputs=x

decoder = Model(latent_inputs,outputs)
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 58)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              60416     
_________________________________________________________________
reshape_1 (Reshape)          (None, 32, 1, 32)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 1, 32)         9248      
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 64, 2, 32)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 2, 64)         18496     
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 128, 4, 64)        0         
__________

Build the discriminators

In [5]:
disc_dim = 256
feat_disc_inputs = Input(shape=(latent_dim,), name='disc_input')
x = feat_disc_inputs
x = Dense(disc_dim, activation='relu')(x)
x = Dense(disc_dim, activation='relu')(x)
feat_disc_outputs = Dense(1,activation='sigmoid')(x)

feat_disc = Model(feat_disc_inputs,feat_disc_outputs,name='feat_disc')
feat_disc.compile(optimizer=Adam(lr=1e-4), 
                      loss="binary_crossentropy")
feat_disc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
disc_input (InputLayer)      (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               13056     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 79,105
Trainable params: 79,105
Non-trainable params: 0
_________________________________________________________________


In [6]:
disc_dim = 256
class_disc_inputs = Input(shape=(n_classes,), name='disc_input')
x = class_disc_inputs
x = Dense(disc_dim, activation='relu')(x)
x = Dense(disc_dim, activation='relu')(x)
class_disc_outputs = Dense(1,activation='sigmoid')(x)

class_disc = Model(class_disc_inputs,class_disc_outputs,name='class_disc')
class_disc.compile(optimizer=Adam(lr=1e-4), 
                      loss="binary_crossentropy")
class_disc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
disc_input (InputLayer)      (None, 8)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               2304      
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 68,353
Trainable params: 68,353
Non-trainable params: 0
_________________________________________________________________


Define composite models

In [7]:
outputs = decoder(Concatenate(axis=-1)(encoder(inputs)))
ae = Model(inputs, outputs, name='ae')
ae.compile(optimizer=Adam(lr=1e-4), 
           loss="binary_crossentropy")
ae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 8, 1)    0                                            
__________________________________________________________________________________________________
encoder (Model)                 [(None, 50), (None,  78266       input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 58)           0           encoder[1][0]                    
                                                                 encoder[1][1]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 256, 8, 1)    125665      concatenate_1[0][0]              
Total para

In [8]:
feat_disc_output = feat_disc(encoder(inputs)[0])
enc_feat_disc = Model(inputs,feat_disc_output,name='enc_feat_disc')
enc_feat_disc.compile(optimizer=Adam(lr=1e-4), 
                      loss="binary_crossentropy")
enc_feat_disc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 8, 1)         0         
_________________________________________________________________
encoder (Model)              [(None, 50), (None, 8)]   78266     
_________________________________________________________________
feat_disc (Model)            (None, 1)                 79105     
Total params: 157,371
Trainable params: 157,371
Non-trainable params: 0
_________________________________________________________________


In [9]:
class_disc_output = class_disc(encoder(inputs)[1])
enc_class_disc = Model(inputs,class_disc_output,name='enc_class_disc')
enc_class_disc.compile(optimizer=Adam(lr=1e-4), 
                      loss="binary_crossentropy")
enc_class_disc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 8, 1)         0         
_________________________________________________________________
encoder (Model)              [(None, 50), (None, 8)]   78266     
_________________________________________________________________
class_disc (Model)           (None, 1)                 68353     
Total params: 146,619
Trainable params: 146,619
Non-trainable params: 0
_________________________________________________________________


define sampling procedures for the latent feature and class distributions

In [29]:
def sample_classes(labels,n_classes=None):
    if n_classes == None:
        n_classes = np.max(labels)
    ulabel = labels == -1
    labels[ulabel] = np.random.randint(0,n_classes,np.sum(ulabel))
    labels = labels.astype(int)
    oh = np.zeros((labels.shape[0],n_classes),dtype=int)
    oh[range(labels.shape[0]),labels]=1
    return oh

def sample_features(n_samples,n_dimensions):
    return np.random.multivariate_normal(np.zeros(n_dimensions),np.eye(n_dimensions),n_samples)

In [25]:
n_samples = 10000
X = np.random.uniform(0,1,(n_samples,)+input_shape)
X_train, X_test = train_test_split(X,train_size=0.5)

In [32]:
epochs = 1
batch_size = 1000
for i_epoch in range(epochs):
    np.random.shuffle(X_train)
    
    for i_batch in range(int(X_train.shape[0]/batch_size)):
        settrainable(ae, True)
        settrainable(encoder, True)
        settrainable(decoder, True)
        
        batch = X_train[i_batch*batch_size:(i_batch+1)*batch_size]
 
        # first perform reconstruction training
        ae.train_on_batch(batch,batch)
        
        # now perform regularization training
        settrainable(feat_disc, True)
        batch_features = encoder.predict(batch)[0]
        fake_features = sample_features(batch_size,latent_dim,)
        
        # now train the feat_disc giving it ones for true, and 
        #     zeros for fake
        discbatch_x = np.concatenate([batch_features,fake_features])
        discbatch_y = np.concatenate([np.ones(batch_size),
                                      np.zeros(batch_size)])
        feat_disc.train_on_batch(discbatch_x,discbatch_y)
        
        # now train the enc_feat_disc but only update the 
        #     encoder weights and try to fool the discriminator
        settrainable(enc_feat_disc, True)
        settrainable(encoder, True)
        settrainable(feat_disc, False)
#         enc_feat_disc.train_on_batch(batch, np.ones(batch_size))
        
        # now perform semi-supervised training
        settrainable(class_disc, True)
        batch_classes = encoder.predict(batch)[1]
        fake_classes = sample_classes(np.ones(batch_size)*-1,n_classes,)
        
        # now train the class_disc giving it ones for true, and 
        #     zeros for fake
        discbatch_x = np.concatenate([batch_classes,fake_classes])
        discbatch_y = np.concatenate([np.ones(batch_size),
                                      np.zeros(batch_size)])
        class_disc.train_on_batch(discbatch_x,discbatch_y)
        
        # now train the enc_class_disc but only update the 
        #     encoder weights and try to fool the discriminator
        settrainable(enc_class_disc, True)
        settrainable(encoder, True)
        settrainable(class_disc, False)
#         enc_class_disc.train_on_batch(batch, np.ones(batch_size))
        
        print("Reconstruction Loss:", 
                  ae.evaluate(X_train, X_train, verbose=0))
        
        print("Feature Adversarial Loss:", 
                  enc_feat_disc.evaluate(X_train, 
                                         np.ones(X_train.shape[0]),
                                         verbose=0))
        print("Class Adversarial Loss:", 
                  enc_class_disc.evaluate(X_train, 
                                          np.ones(X_train.shape[0]),
                                          verbose=0))

Reconstruction Loss: 2.1366917144775393
Feature Adversarial Loss: 0.7031831353187561
Class Adversarial Loss: 0.6886258508682251
Reconstruction Loss: 2.0890271286010744
Feature Adversarial Loss: 0.7029897987365723
Class Adversarial Loss: 0.6881212435722351
Reconstruction Loss: 2.045940550994873
Feature Adversarial Loss: 0.7028387783050537
Class Adversarial Loss: 0.687585341835022
Reconstruction Loss: 2.006192244911194
Feature Adversarial Loss: 0.7026996500968933
Class Adversarial Loss: 0.6870428886413574
Reconstruction Loss: 1.9689696153640748
Feature Adversarial Loss: 0.7025250971794128
Class Adversarial Loss: 0.686519391822815
